# Tail Assignment Problem - Lecture Notebook

This notebook demonstrates the step-by-step execution of the **Tail Assignment Problem** using Python.
The problem involves assigning aircraft to flights while minimizing operational costs and adhering to constraints such as ground time, forbidden destinations, and maintenance requirements.

In [ ]:
# Step 1: Import necessary libraries
import numpy as np
import pandas as pd
import datetime as dt
import time

import re
!pip install pulp
import pulp
import numpy as np
import pandas as pd
import itertools
from copy import deepcopy

import plotly.express as px
import plotly.graph_objects as go

import warnings
import pandas as pd

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Suppress FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 35.2 MB/s eta 0:00:00


## Step 1: Define Fleet and Aircraft Classes

The `Fleet` and `Aircraft` classes are defined to represent the fleet of aircraft and their attributes.
These classes provide methods to retrieve information such as fuel burn, seat capacity, and forbidden destinations for each aircraft.

In [ ]:
# Functions and Classes for fleet

class Fleet:
    """
    Represents a fleet of aircraft.

    Attributes:
        aircrafts (list): A list of Aircraft objects in the fleet.
    """
    def __init__(self, aircraft_dataframe):
        """Initialize a Fleet object with an empty list of aircraft."""
        self.aircraft_dataframe = aircraft_dataframe
        self.aircraft = [Aircraft(x[1]) for x in aircraft_dataframe.iterrows()]

    def add_aircraft(self, aircraft):
        """
        Add an Aircraft object to the fleet.

        Args:
            aircraft (Aircraft): The Aircraft object to add to the fleet.
        """
        self.aircrafts.append(aircraft)

    def get_all_registrations(self):
        """
        Get a list of all registration numbers in the fleet.

        Returns:
            list: A list of registration numbers of all aircraft in the fleet.
        """
        return [aircraft.registration for aircraft in self.aircraft]

    def get_ac_fleet_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective aircraft types.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values are aircraft types.
        """
        return {x.registration: x.type for x in self.aircraft}

    def get_ac_fuel_burn_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective fuel burn.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values is the fuel born of the specific aircraft registration.
        """
        return {x.registration: x.fuel_burn for x in self.aircraft}

    def get_ac_mtow_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective mtow.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values is the mtow of the specific aircraft registration.
        """
        return {x.registration: x.mtow for x in self.aircraft}

    def get_ac_seat_capacity_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective seat capacity.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values is the seat capacity of the specific aircraft registration.
        """
        return {x.registration: x.seat_capacity for x in self.aircraft}

    def get_ac_min_ground_time_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective minimum ground time.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values is the minimum ground time of the specific aircraft registration.
        """
        return {x.registration: x.min_gnd for x in self.aircraft}

    def get_ac_loc_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective location at start.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values is the location at start of the specific aircraft registration.
        """
        return {x.registration: x.loc for x in self.aircraft}


    def get_ac_forbidden_destination_dict(self):
        """
        Returns a dictionary mapping aircraft registrations to their respective forbidden destination.

        Returns:
            dict: A dictionary where keys are aircraft registrations and values is the forbidden destination of the specific aircraft registration.
        """
        return {x.registration: x.forbidden_dest for x in self.aircraft}


    def get_forbidden_destination(self, registration):
        forbidden_dest_dict = self.get_ac_forbidden_destination_dict()

        return forbidden_dest_dict[registration]


    def get_fleet_ac_dict(self):
        """
        Returns a dictionary mapping aircraft types to lists of their respective registrations.

        Uses the get_ac_fleet_dict method internally to obtain the mapping of registrations to aircraft types.

        Returns:
            dict: A dictionary where keys are aircraft types and values are lists of aircraft registrations.

        """
        ac_fleet = self.get_ac_fleet_dict()
        fleet_ac = {}
        for key, value in ac_fleet.items():
            if value not in fleet_ac:
                fleet_ac[value] = [key]
            else:
                fleet_ac[value].append(key)
        return fleet_ac

    def get_registrations_from_type(self, ac_types):
        """
        Returns a list of all aircraft registrations from the given type(s).

        Returns:
            list: A list with all aircraft registrations of given type(s).
        """
        if type(ac_types) != list:
            ac_types = [ac_types]
        return [x.registration for x in self.aircraft if x.type in ac_types]


class Aircraft:
    """
    Represents an individual aircraft.

    Attributes:
        registration (str): The registration number of the aircraft.
        type (str): The subtype of the aircraft.
        seat_capacity (int): The seating capacity of the aircraft.
        fuel_burn (float): The fuel burn rate of the aircraft.
        forbidden_dest (list): A list of forbidden destinations for the aircraft.
        loc (str): The current location of the aircraft.
        mtow (float): The maximum takeoff weight of the aircraft.
        min_gnd (int): The minimum ground time required for the aircraft.
        first_mx_hub (str): The first maintenance hub for the aircraft (default is None).
    """
    def __init__(self, aircraft_dataframe):
        """
        Initialize an Aircraft object.

        Args:
            aircraft_dataframe (pd.Series): A pandas Series containing aircraft data.
        """
        self.registration = aircraft_dataframe['aircraft_registration']
        self.type = aircraft_dataframe['aircraft_sub_type']
        self.seat_capacity = aircraft_dataframe['seat_capacity']
        self.fuel_burn = aircraft_dataframe['fuel_burn']
        self.forbidden_dest = aircraft_dataframe['forbidden_dest']
        self.loc = aircraft_dataframe['location']
        self.mtow = aircraft_dataframe['mtow']
        self.min_gnd = aircraft_dataframe['min_gnd_time']
        self.first_mx_hub = None


def get_aircraft_dataframe(flight_schedule):
    """
    Create a dataframe containing aircraft information from the flight schedule.

    Args:
        flight_schedule (pd.DataFrame): A dataframe containing the flight schedule.

    Returns:
        pd.DataFrame: A dataframe enriched with aircraft information, including registration,
                      type, seat capacity, fuel burn, forbidden destinations, location, mtow,
                      and minimum ground time.
    """
    # Create dataframe from given aircrafts in flight_schedule
    aircraft_dataframe = flight_schedule[['aircraft_registration', 'aircraft_sub_type']
                                ].value_counts().reset_index()[
                                    ['aircraft_registration', 'aircraft_sub_type']
                                    ].sort_values(by='aircraft_registration'
                                                ).reset_index(drop=True)
    # Join with info of fleet to enrich dataframe
    aircraft_dataframe = aircraft_dataframe.join(
        pd.DataFrame(aircraft_dataframe['aircraft_sub_type'].map(
            get_fleet_info()).to_list(), index=aircraft_dataframe.index))

    # Get the last location of each aircraft and add it
    filtered_df = flight_schedule.sort_values('departure_datetime_schedule_utc_dis')
    last_locations = pd.DataFrame(filtered_df.groupby('aircraft_registration')
                                    ['departure_airport_actual'].first()).reset_index()
    aircraft_dataframe = pd.merge(aircraft_dataframe,
                                last_locations,
                                on='aircraft_registration',
                                how='left').rename(columns={'departure_airport_actual': 'location'})

    # If no location found, set to ZRH
    aircraft_dataframe.loc[aircraft_dataframe['location'].isnull(), 'location'] = 'ZRH'

    return aircraft_dataframe.reset_index(drop=True)


def get_fleet_info():
    """
    Retrieve fleet information for different aircraft subtypes.

    Returns:
        dict: A dictionary where keys are aircraft subtypes and values are dictionaries
              containing attributes like fuel burn, forbidden destinations, seat capacity,
              mtow, and minimum ground time.
    """
    fleet_info = {
        '221': {'fuel_burn': 700,
                 'forbidden_dest': [],
                 'seat_capacity': 125,
                 'mtow': 58,
                 'min_gnd_time': 30},
        '223': {'fuel_burn': 800,
                 'forbidden_dest': ['LCY', 'FLR'],
                 'seat_capacity': 145,
                 'mtow': 70.9,
                 'min_gnd_time': 30},
        '777': {'fuel_burn': 2500,
                 'forbidden_dest': [],
                 'seat_capacity': 320,
                 'mtow': 351.5,
                 'min_gnd_time': 60},
        }
    return fleet_info



## Step 2: Define Cost Dataframes

Cost dataframe are defined to calculate the operating costs for flights and ground costs for aircraft.
These include fuel costs, seat costs, cancellation costs, and airport ground costs.

In [ ]:
def get_operating_cost(fleet, flight_schedule, input):
    """
    Generate the flight_cost DataFrame based on the fleet, flight_schedule, and input.

    Args:
        fleet (Fleet): The Fleet object containing aircraft information.
        flight_schedule (pd.DataFrame): The flight schedule DataFrame.
        input (dict): Input dictionary containing cost parameters.

    Returns:
        pd.DataFrame: A DataFrame containing operating costs for each flight, including fuel cost,
                      seat cost, and cancellation cost.
    """
    # Explode the possible_aircraft column to create one row per aircraft per flight
    flight_cost = flight_schedule.explode('possible_aircraft')

    # Calculate fuel cost based on flight duration and fuel burn rate
    flight_cost['fuel_cost'] = (
        flight_cost['flight_duration_dec_hour']
        * flight_cost['possible_aircraft'].map(fleet.get_ac_fuel_burn_dict())
        * input['cost']['C_fuel']
    )

    # Calculate empty seats and seat cost
    flight_cost['empty_seats'] = (
        flight_cost['possible_aircraft'].map(fleet.get_ac_seat_capacity_dict())
        - flight_cost['total_pax']
    )
    flight_cost['seat_cost'] = flight_cost['empty_seats'].astype(float)  # Explicitly cast to float
    flight_cost.loc[flight_cost['empty_seats'] > 0, 'seat_cost'] *= input['cost']["C_empty_seat"]
    flight_cost.loc[flight_cost['empty_seats'] <= 0, 'seat_cost'] *= (
        input['cost']["C_overbooked_seat"] * -1
    )

    # Add cancellation cost
    flight_cost['cancel_cost'] = input['cost']["C_cancel"]

    return flight_cost


def get_ground_cost(fleet, input, airports):
    """
    Generate the ground_cost DataFrame based on the fleet, input, and airports.

    Args:
        fleet (Fleet): The Fleet object containing aircraft information.
        input (dict): Input dictionary containing cost parameters and hub airports.
        airports (list): List of all airports in the flight schedule.

    Returns:
        pd.DataFrame: A DataFrame containing ground costs for each aircraft at each airport,
                      including MTOW-based costs and fixed costs for hub airports.
    """
    # Create a DataFrame with all aircraft registrations
    ground_cost = pd.DataFrame(
        data=fleet.get_all_registrations(),
        columns=['aircraft_registration']
    )

    # Add MTOW (Maximum Takeoff Weight) for each aircraft
    ground_cost['mtow'] = ground_cost['aircraft_registration'].map(fleet.get_ac_mtow_dict())

    # Add forbidden destinations for each aircraft
    forbidden_dest_dict = fleet.get_ac_forbidden_destination_dict()
    ground_cost['airport'] = ground_cost['aircraft_registration'].apply(
        lambda reg: [airport for airport in airports if airport not in forbidden_dest_dict[reg]]
    )

    # Explode the airport column to create one row per aircraft per airport
    ground_cost = ground_cost.explode('airport').reset_index(drop=True)

    # Calculate airport cost based on MTOW
    ground_cost['airport_cost'] = ground_cost['mtow'] * input['cost']['C_ground']

    # Set cost for maintenance airports to a fixed value
    hub_airports = ['ZRH', 'GVA']
    ground_cost.loc[
        ground_cost['airport'].isin(hub_airports),
        'airport_cost'
    ] = 10

    return ground_cost

## Step 3: Define the Tail Assignment Problem (TAProblem) Class

The `TAProblem` class encapsulates the tail assignment problem.
It includes methods to set up decision variables, constraints, and the objective function for the optimization model.
The constraints ensure that flights are either flown or canceled, flow balance is maintained, and ground time and forbidden airport rules are respected.

In [ ]:
def get_pulp_dict(data, key, value):
    """
    Creates a dictionary by zipping two columns from a given dataset.

    Args:
        data (DataFrame): The dataset containing the columns.
        key (str): The column to use as keys.
        value (str): The column to use as values.

    Returns:
        dict: A dictionary mapping keys to values.
    """
    return dict(zip(data[key], data[value]))

def flatten(xss):
    """
    Flattens a list of lists into a single list.

    Args:
        xss (list of lists): The list of lists to flatten.

    Returns:
        list: A flattened list.
    """
    return [x for xs in xss for x in xs]


class TAProblem:
    """
    Represents the Tail Assignment Problem (TAP) for aircraft scheduling.

    Attributes:
        tol (float): Tolerance for numerical calculations.
        minutes_discretization (int): Time discretization in minutes.
        rng (RandomState): Random number generator.
        input (dict): Input data for the problem.
        registration_date (list): Aircraft registration and date range.
        flight_schedule (DataFrame): Flight schedule data.
        flight_cost (DataFrame): Flight cost data.
        ground_cost (DataFrame): Ground cost data.
        var_dict (dict): Dictionary of decision variables.
        fleet (object): Fleet information.
        mode (str): Mode of the model (e.g., 'extensive').
    """

    def __init__(self, input, fleet, flight_schedule, flight_cost, ground_cost, rng=42):
        """
        Initializes the TAProblem instance.

        Args:
            input (dict): Input data for the problem.
            fleet (object): Fleet information.
            flight_schedule (DataFrame): Flight schedule data.
            flight_cost (DataFrame): Flight cost data.
            ground_cost (DataFrame): Ground cost data.
            rng (int, optional): Random seed. Defaults to 42.
        """
        self.tol = 1e-6
        self.minutes_discretization = 15
        self.rng = np.random.RandomState(seed=rng)
        self.input = deepcopy(input)

        self.registration_date = [input['aircraft_registration_selection'], [input['start_date'], input['end_date']]]
        self.flight_schedule = flight_schedule
        self.flight_cost = flight_cost
        self.ground_cost = ground_cost

        self.var_dict = dict()
        self.fleet = fleet

        self._setup_variables_ta()
        self._setup_variables_general()

    def reset(self, input, fleet, flight_schedule, flight_cost, ground_cost, rng=42):
        """
        Resets the TAProblem instance with new data.

        Args:
            input (dict): Input data for the problem.
            fleet (object): Fleet information.
            flight_schedule (DataFrame): Flight schedule data.
            flight_cost (DataFrame): Flight cost data.
            ground_cost (DataFrame): Ground cost data.
            rng (int, optional): Random seed. Defaults to 42.
        """
        self.__init__(input, fleet, flight_schedule, flight_cost, ground_cost, rng)

    def _make_extensive_model(self):
        """
        Formulates the two-stage extensive form of the problem.

        Returns:
            LpProblem: The extensive form model.
        """
        self.mode = 'extensive'

        model = pulp.LpProblem("TailAssignment", pulp.LpMinimize)
        model = self._setup_decision_variable_ta(model)
        model = self._setup_constraints_ta(model)
        model = self._setup_objectives(model)

        return model

    def _setup_variables_general(self):
        """
        Sets up general variables for the problem.
        """
        self.aircraft_time = list(
            itertools.product(self.registration_date[0], self.time_dis_int_list))

    def _setup_variables_ta(self):
        """
        Sets up variables specific to the Tail Assignment Problem.
        """
        self.flight_number_aircraft = list(zip(self.flight_cost['flight_identifier'],
                                              self.flight_cost['possible_aircraft']))

        self.flight_number = self.flight_schedule['flight_identifier'].to_list()

        self.time_dis_int_list = list(range(int((self.input['end_date'] - self.input['start_date']
                           ).total_seconds() / 60
                          / self.minutes_discretization)))

        self.airports_in_flight_schedule = [x for x in np.unique(self.ground_cost['airport'])]

        self.aircraft_airport_time = [(aircraft, airport, time)
                                    for aircraft in [x for x in np.unique(self.ground_cost['aircraft_registration'])]
                                    for airport in self.airports_in_flight_schedule
                                    for time in self.time_dis_int_list]

    def _make_model(self):
        """
        Creates a basic optimization model.

        Returns:
            LpProblem: The optimization model.
        """
        model = pulp.LpProblem("TailAssignment", pulp.LpMinimize)
        return model

    def _setup_decision_variable_ta(self, model):
        """
        Sets up decision variables for the Tail Assignment Problem.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with decision variables.
        """
        self.var_dict['F_p_f'] = pulp.LpVariable.dicts("F_p_f",
                                                      self.flight_number_aircraft,
                                                      cat=pulp.LpBinary)
        self.var_dict['C_f'] = pulp.LpVariable.dicts("C_f",
                                                    self.flight_number,
                                                    cat=pulp.LpBinary)
        self.var_dict['G_p_n_t'] = pulp.LpVariable.dicts("G_p_n_t",
                                                        self.aircraft_airport_time,
                                                        cat=pulp.LpBinary)

        return model

    def _setup_constraints_ta(self, model):
        """
        Sets up constraints for the Tail Assignment Problem.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with constraints.
        """
        model = self._setup_c2_ta_flight_cancelation(model)
        model = self._setup_c3_flowbalance(model)
        model = self._setup_c4_ta_ground_time(model)
        model = self._setup_c5_ta_forbidden_airports(model)

        return model

    def _setup_c2_ta_flight_cancelation(self, model):
        """
        Sets up flight cancellation constraints.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with flight cancellation constraints.
        """
        for _, flight_row in self.flight_schedule.iterrows():
            flight_id = flight_row['flight_identifier']
            aircraft_possible = flight_row['possible_aircraft']

            name_c2 = f"fly_or_cancel_1[{flight_id}]"
            model += (pulp.lpSum(self.var_dict['F_p_f'][(flight_id, aircraft)] for
                            aircraft in aircraft_possible) \
                    + self.var_dict['C_f'][flight_id] == 1,
                name_c2)

        return model

    def _setup_c3_flowbalance(self, model):
        """
        Sets up flow balance constraints.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with flow balance constraints.
        """
        aicraft_location_start = self.fleet.get_ac_loc_dict()
        for aircraft, airport, time in self.aircraft_airport_time:
            flight_id_departure = self.flight_schedule.loc[
                (self.flight_schedule['possible_aircraft'].apply(lambda x: aircraft in x))
                & (self.flight_schedule['departure_datetime_schedule_uts_dis_int'] == time)
                & (self.flight_schedule['departure_airport_actual'] == airport),
                'flight_identifier'].to_list()
            flight_id_arrival = self.flight_schedule.loc[
                (self.flight_schedule['possible_aircraft'].apply(lambda x: aircraft in x))
                & (self.flight_schedule['arrival_datetime_schedule_utc_dis_int'] == time)
                & (self.flight_schedule['arrival_airport_actual'] == airport),
                'flight_identifier'].to_list()

            f_o_1 = pulp.lpSum(self.var_dict['F_p_f'][(flight_id, aircraft)]
                              for flight_id in flight_id_departure)
            f_t_1 = pulp.lpSum(self.var_dict['F_p_f'][(flight_id, aircraft)]
                              for flight_id in flight_id_arrival)

            name_c3 = f"node_1[{aircraft},{airport},{time}]"
            if airport == aicraft_location_start[aircraft] and time == 0:
                model += (
                    f_o_1 - f_t_1 + self.var_dict['G_p_n_t'][(aircraft, airport, time)] == 1,
                    name_c3)
            elif airport != aicraft_location_start[aircraft] and time == 0:
                model += (
                    f_o_1 - f_t_1 + self.var_dict['G_p_n_t'][(aircraft, airport, time)] == 0,
                    name_c3)
            else:
                model += (
                    f_o_1 - f_t_1
                    + self.var_dict['G_p_n_t'][(aircraft, airport, time)]
                    - self.var_dict['G_p_n_t'][(aircraft, airport, (time - 1))] == 0,
                    name_c3)

        return model

    def _setup_c4_ta_ground_time(self, model):
        """
        Sets up ground time constraints.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with ground time constraints.
        """
        for flight, aircraft in self.flight_number_aircraft:
            name_c4 = f"min_gnd_pf[{aircraft},{flight}]_c4"

            dis_time_arrival = int(self.flight_schedule.loc[
                self.flight_schedule['flight_identifier']==flight,
                'arrival_datetime_schedule_utc_dis_int'].values[0])
            arrival_airport = self.flight_schedule.loc[
                self.flight_schedule['flight_identifier']==flight,
                'arrival_airport_actual'].values[0]

            min_gnd_time_dis = int(self.fleet.get_ac_min_ground_time_dict()[aircraft] / self.minutes_discretization)

            if dis_time_arrival <= max(self.time_dis_int_list) - min_gnd_time_dis:
                time_range = range(dis_time_arrival, (dis_time_arrival + min_gnd_time_dis))
                g_t_1 = pulp.lpSum(
                    self.var_dict['G_p_n_t'][(aircraft, arrival_airport, t)]
                    for t in time_range)

                M = 1000
                model += (g_t_1 >= min_gnd_time_dis -
                         M * (1 - self.var_dict['F_p_f'][(flight, aircraft)]), name_c4 + "_1")
                model += (g_t_1 <= min_gnd_time_dis +
                         M * (1 - self.var_dict['F_p_f'][(flight, aircraft)]), name_c4 + "_2")

            else:
                time_range = range(dis_time_arrival, max(self.time_dis_int_list))
                g_t_1 = pulp.lpSum(
                    self.var_dict['G_p_n_t'][(aircraft, arrival_airport, t)]
                    for t in time_range)

                expected_time = max(self.time_dis_int_list) - dis_time_arrival - 1
                M = 1000
                model += (g_t_1 >= expected_time -
                         M * (1 - self.var_dict['F_p_f'][(flight, aircraft)]), name_c4 + "_1")
                model += (g_t_1 <= expected_time +
                         M * (1 - self.var_dict['F_p_f'][(flight, aircraft)]), name_c4 + "_2")

        return model

    def _setup_c5_ta_forbidden_airports(self, model):
        """
        Sets up forbidden airport constraints.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with forbidden airport constraints.
        """
        forbidden_exists = any([x for x in list(set(flatten([aircraft.forbidden_dest for
                                                aircraft in self.fleet.aircraft])))
                              if x in self.airports_in_flight_schedule])

        if forbidden_exists:
            for aircraft in self.fleet.aircraft:
                for airport in aircraft.forbidden_dest:
                    name_c5 = f'forbidden_airports[{aircraft},{airport}]'
                    try:
                        model += (
                            pulp.lpSum(self.var_dict['G_p_n_t'][(aircraft.registration,
                                                               airport,
                                                               t)]
                                      for t in self.time_dis_int_list) == 0, name_c5)
                    except KeyError:
                        pass

        return model

    def _setup_objectives(self, model):
        """
        Sets up the objective function for the problem.

        Args:
            model (LpProblem): The optimization model.

        Returns:
            LpProblem: The updated model with the objective function.
        """
        operating_cost_dict = dict(zip([(f, a) for f, a in zip(self.flight_cost['flight_identifier'],
                                     self.flight_cost['possible_aircraft'])],
                                self.flight_cost['fuel_cost']))
        seat_cost_dict = dict(zip([(f, a) for f, a in zip(self.flight_cost['flight_identifier'],
                                 self.flight_cost['possible_aircraft'])],
                                self.flight_cost['seat_cost']))
        cancel_cost_dict = dict(zip(self.flight_schedule['flight_identifier'],
                                [self.input['cost']['C_cancel']] * len(self.flight_schedule)))

        ground_cost_mtow_dict = {(x[0], x[1], n): x[2] for _, x in
                               self.ground_cost[
                                   ['aircraft_registration', 'airport', 'airport_cost']
                                   ].iterrows() for n in self.time_dis_int_list}

        operating_cost = pulp.lpSum(operating_cost_dict[(flight_id, aircraft)]
                                   * self.var_dict['F_p_f'][(flight_id, aircraft)]
                                   + seat_cost_dict[(flight_id, aircraft)]
                                   * self.var_dict['F_p_f'][(flight_id, aircraft)]
                                   for flight_id, aircraft in self.flight_number_aircraft)
        cancel_cost = pulp.lpSum(cancel_cost_dict[flight_id]
                               * self.var_dict['C_f'][flight_id]
                               for flight_id in self.flight_number)

        airport_ground_cost = pulp.lpSum(ground_cost_mtow_dict[aircraft, airport, time]
                                       * self.var_dict['G_p_n_t'][(aircraft, airport, time)]
                                       for aircraft, airport, time in
                                       self.aircraft_airport_time if airport
                                       not in self.fleet.get_forbidden_destination(aircraft))

        model += operating_cost + cancel_cost + airport_ground_cost

        return model

    def solve_extensive(self, gap=0.02, time_limit=600, threads=1):
        """
        Solves the extensive form of the problem.

        Args:
            gap (float, optional): Relative gap for the solver. Defaults to 0.02.
            time_limit (int, optional): Time limit for the solver in seconds. Defaults to 600.
            threads (int, optional): Number of threads for the solver. Defaults to 1.

        Returns:
            LpProblem: The solved model.
        """
        model = self._make_extensive_model()
        self.ef_solving_results = {'primal': [], 'dual': [], 'incumbent': [], 'time': []}
        pulp.PULP_CBC_CMD(threads=threads, timeLimit=time_limit, gapRel=gap, msg=True).solve(model)
        return model


## Step 4: Define Plotting Functions

Plotting functions are defined to visualize the results of the tail assignment problem.
These include Gantt charts for aircraft schedules and flight plans, showing flown and canceled flights.

In [ ]:
def plot_aircraft_schedules_plotly(input, df_flight_flown, file_add=""):
    """
    Plots a Gantt chart of aircraft schedules using Plotly.

    Args:
        input (dict): A dictionary containing configuration input, including:
            - 'aircraft_registration_selection': List of selected aircraft registrations.
            - 'start_date': Start date of the schedule.
            - 'end_date': End date of the schedule.
        df_flight_flown (pd.DataFrame): DataFrame containing flight data with columns:
            - 'departure_datetime_schedule_utc_dis': Scheduled departure datetime (UTC).
            - 'arrival_datetime_schedule_utc_dis': Scheduled arrival datetime (UTC).
            - 'departure_airport_actual': Departure airport code.
            - 'arrival_airport_actual': Arrival airport code.
            - 'aircraft_selected': Aircraft registration.
        file_add (str, optional): Additional string to append to the filename. Defaults to "".

    Returns:
        None: Displays the Gantt chart using Plotly.
    """
    # Add underscore at beginning if not already there
    if len(file_add) > 0 and file_add[0] != "_":
        file_add = "_" + file_add

    filename = "schedule_plot_{}ac_{}days{}".format(
        len(input['aircraft_registration_selection']),
        (input['end_date'] - input['start_date']).days,
        file_add
    )

    df_flight_flown = df_flight_flown.sort_values(by='aircraft_selected', ascending=True)

    pdf_ff = df_flight_flown.rename(columns={'departure_datetime_schedule_utc_dis': 'begin_dt',
                                             'arrival_datetime_schedule_utc_dis': 'end_dt'})
    pdf_ff['text'] = pdf_ff['departure_airport_actual'] + "-" + pdf_ff['arrival_airport_actual']
    pdf_ff['source'] = 'flight'

    pdf = pdf_ff
    pdf['source_bool'] = pdf['source'] == 'flight'

    # Create a color map for True (green) and False (red)
    color_map = {'flight': "lightskyblue"}

    # Customize hover templates
    hover_template_flight = (
        'Flight identifier: %{customdata[0]}<br>' +
        'Flight duration: %{customdata[1]:.2f}<br>' +
        'Total passengers: %{customdata[2]}<br>'
    )

    # Plot Gantt chart using Plotly
    fig = px.timeline(pdf.sort_values(by=['aircraft_selected', 'departure_airport_actual', 'arrival_airport_actual']),
                      x_start="begin_dt",
                      x_end="end_dt",
                      y="aircraft_selected",
                      title=filename,
                      text='text',
                      color_discrete_map=color_map,
                      color="source",
                      range_x=[input['start_date'], input['end_date']]
                      )

    # Add customdata for hover information
    fig.update_traces(
        customdata=pdf[['flight_identifier', 'flight_duration_dec_hour', 'total_pax', 'text']].values,
        textangle=90,
        textposition='inside'
    )

    # Apply hover templates conditionally
    for trace in fig.data:
        if trace.name == 'flight':
            trace.hovertemplate = hover_template_flight

    # Update text orientation to be vertical
    fig.update_traces(textangle=90, textposition='inside')
    fig.update_yaxes(categoryorder="total ascending")
    fig.show()


def plot_flightplan_plotly(input, df_flight_flown, df_flight_canceled, file_add="", colormap='Set3'):
    """
    Plots a flight plan with normal and canceled flights using Plotly.

    Args:
        input (dict): A dictionary containing configuration input, including:
            - 'aircraft_registration_selection': List of selected aircraft registrations.
            - 'start_date': Start date of the flight plan.
            - 'end_date': End date of the flight plan.
        df_flight_flown (pd.DataFrame): DataFrame containing flown flight data with columns:
            - 'departure_datetime_schedule_utc_dis': Scheduled departure datetime (UTC).
            - 'arrival_datetime_schedule_utc_dis': Scheduled arrival datetime (UTC).
            - 'departure_airport_actual': Departure airport code.
            - 'arrival_airport_actual': Arrival airport code.
            - 'aircraft_selected': Aircraft registration.
        df_flight_canceled (pd.DataFrame): DataFrame containing canceled flight data with columns:
            - 'departure_datetime_schedule_utc_dis': Scheduled departure datetime (UTC).
            - 'arrival_datetime_schedule_utc_dis': Scheduled arrival datetime (UTC).
            - 'departure_airport_actual': Departure airport code.
            - 'arrival_airport_actual': Arrival airport code.
        file_add (str, optional): Additional string to append to the filename. Defaults to "".
        colormap (str, optional): Colormap to use for aircraft encoding. Defaults to 'Set3'.

    Returns:
        None: Displays the flight plan using Plotly.
    """
    df_flight_flown = df_flight_flown.sort_values(by='aircraft_selected', ascending=True)

    # Add underscore at beginning if not already there
    if len(file_add) > 0 and file_add[0] != "_":
        file_add = "_" + file_add

    filename = "flightplan_plot_{}ac_{}days{}".format(
        len(input['aircraft_registration_selection']),
        (input['end_date'] - input['start_date']).days,
        file_add
    )

    # Prep aircraft encoding
    unique_aircraft = list(set(df_flight_flown['aircraft_selected'].unique()))
    unique_aircraft.sort()
    aircraft_color_encoding = {aircraft: px.colors.qualitative.Set3[i % len(px.colors.qualitative.Set3)] for i, aircraft in enumerate(unique_aircraft)}

    # Prep airport encoding
    unique_airport = list(
        set(df_flight_flown['departure_airport_actual'].unique()).union(
            set(df_flight_flown['arrival_airport_actual'].unique())).union(
                set(df_flight_canceled['departure_airport_actual'].unique())).union(
                    set(df_flight_canceled['arrival_airport_actual'].unique()))
    )

    unique_airport = [x for x in unique_airport if type(x) == str]
    unique_airport.sort(reverse=True)
    airport_row_encoding = {airport: number for number, airport in enumerate(unique_airport)}

    # Prep normal lines of flight
    flight_lines = []
    for aircraft in unique_aircraft:
        try:
            df_flight_flown_temp = df_flight_flown[df_flight_flown['aircraft_selected'] == aircraft].sort_values(by='departure_datetime_schedule_utc_dis')
            flight_lines.append((input['start_date'], airport_row_encoding[df_flight_flown_temp.iloc[0]['departure_airport_actual']], aircraft))

            for _, row in df_flight_flown_temp.iterrows():
                flight_lines.append((row['departure_datetime_schedule_utc_dis'], airport_row_encoding[row['departure_airport_actual']], aircraft))
                flight_lines.append((row['arrival_datetime_schedule_utc_dis'], airport_row_encoding[row['arrival_airport_actual']], aircraft))

            flight_lines.append((input['end_date'], airport_row_encoding[df_flight_flown_temp.iloc[-1]['arrival_airport_actual']], aircraft))
        except IndexError:
            continue

    # Prep canceled flights
    canceled_flights_lines = []
    for _, row in df_flight_canceled.iterrows():
        canceled_flights_lines.append((row['departure_datetime_schedule_utc_dis'], airport_row_encoding[row['departure_airport_actual']], 'Cancelled'))
        canceled_flights_lines.append((row['arrival_datetime_schedule_utc_dis'], airport_row_encoding[row['arrival_airport_actual']], 'Cancelled'))

    # Convert to DataFrame
    flight_lines_df = pd.DataFrame(flight_lines, columns=['datetime', 'airport', 'aircraft'])
    canceled_flights_lines_df = pd.DataFrame(canceled_flights_lines, columns=['datetime', 'airport', 'status'])

    # Plot with Plotly Express
    fig = px.line(flight_lines_df, x='datetime', y='airport', color='aircraft',
                  color_discrete_map=aircraft_color_encoding,
                  title=filename,
                  labels={'datetime': 'Time (month-day hour)', 'airport': 'Airport Code'})

    # Add traces for canceled flights, ensuring they share the same name and legend group
    for i in range(0, len(canceled_flights_lines_df), 2):
        fig.add_trace(go.Scatter(
            x=[canceled_flights_lines_df.iloc[i]['datetime'], canceled_flights_lines_df.iloc[i + 1]['datetime']],
            y=[canceled_flights_lines_df.iloc[i]['airport'], canceled_flights_lines_df.iloc[i + 1]['airport']],
            mode='lines+markers',
            line=dict(color='red', dash='dash'),
            name='Cancelled',  # Ensure all canceled flights share the same name
            legendgroup='Cancelled',  # Group all canceled flights under the same legend entry
            showlegend=(i == 0)  # Show legend only for the first canceled flight
        ))

    # Update layout for readability
    fig.update_layout(
        xaxis=dict(tickformat="%m-%d %H:%M", title="Time (month-day hour)"),
        yaxis=dict(tickvals=[*range(len(unique_airport))], ticktext=unique_airport, title="Airport Code"),
        legend_title="Aircraft",
        font=dict(size=18),
        title_x=0.5
    )
    fig.show()


## Step 5: Extract Results

Functions are defined to extract and analyze the results of the optimization model.
These include extracting flight statistics, such as flown and canceled flights, and generating detailed model statistics.

In [ ]:
def extract_pulp_var_info(pulp_var_string):
    """
    Extracts variable information from a PuLP variable string.

    Args:
        pulp_var_string (str): The string representation of a PuLP variable.

    Returns:
        list: A list containing the variable type and its associated components.
    """
    # Check if the variable is a cancellation variable (C_f)
    if 'C_f' in pulp_var_string:
        temp = ['C_f_', pulp_var_string.replace('C_f_', '')]
    else:
        # Split the variable string to extract components
        temp = pulp_var_string.split('(')

    # Return the variable type and its components as a list
    return [temp[0]] + temp[1].replace(']', '').split(',')


def get_flight_stats(milp_input, flight_schedule):
    """
    Extracts flight statistics from the optimization model.

    Args:
        milp_input (TAProblem): The Tail Assignment Problem instance containing the model and variables.
        flight_schedule (pd.DataFrame): The flight schedule DataFrame.

    Returns:
        tuple: A tuple containing:
            - df_flight_flown (pd.DataFrame): DataFrame of flown flights with selected aircraft.
            - df_flight_canceled (pd.DataFrame): DataFrame of canceled flights.
            - df_flight_counter (pd.DataFrame): DataFrame summarizing flight count (FC) and flight hours (FH) per aircraft.
    """
    # Extract scheduled flights and their assigned aircraft
    if type(milp_input.var_dict['F_p_f']) == dict:
        list_scheduled_flight_aircraft = [
            (key[0], key[1]) for key, value in milp_input.var_dict['F_p_f'].items() if value.value() == 1
        ]
    elif milp_input.var_dict['F_p_f'] is None:
        list_scheduled_flight_aircraft = [(None, None)]
    else:
        # For PuLP version, extract variables directly from the model
        list_scheduled_flight_aircraft = [
            (x.name.split('[')[1].split(',')[0], x.name.split(',')[1].split(']')[0])
            for x in milp_input.model.variables()
            if x.name.startswith('F_p_f') and x.varValue == 1
        ]

    # Extract the list of scheduled flights
    list_scheduled_flights = [x[0] for x in list_scheduled_flight_aircraft]

    # Create DataFrame for flown flights
    df_flight_flown = flight_schedule[flight_schedule['flight_identifier'].isin(list_scheduled_flights)]
    df_flight_flown['aircraft_selected'] = df_flight_flown['flight_identifier'].map(
        {x[0]: x[1] for x in list_scheduled_flight_aircraft}
    )

    # Create DataFrame for canceled flights
    df_flight_canceled = flight_schedule[~flight_schedule['flight_identifier'].isin(list_scheduled_flights)]

    # Summarize flight count (FC) and flight hours (FH) per aircraft
    df_flight_counter = df_flight_flown.groupby('aircraft_selected').agg(
        FC=('flight_identifier', len),
        FH=('flight_duration', np.sum)
    ).reset_index()

    return df_flight_flown, df_flight_canceled, df_flight_counter


def get_model_stats(milp_input, model=None):
    """
    Extracts variable statistics from the optimization model.

    Args:
        milp_input (TAProblem): The Tail Assignment Problem instance containing the model and variables.
        model (pulp.LpProblem, optional): The PuLP optimization model. Defaults to None.

    Returns:
        dict: A dictionary containing DataFrames for each variable type (F_p_f, C_f, G_p_n_t).
    """
    # Define column names for each variable type
    cols_gurobi_var = {
        'F_p_f': ['name', 'flight_number', 'aircraft_registration', 'value'],
        'C_f': ['name', 'flight_number', 'value'],
        'G_p_n_t': ['name', 'aircraft_registration', 'airport', 'time', 'value']
    }

    df_dict = {}

    # Use the provided model or the model from the TAProblem instance
    if model is None:
        model = milp_input.model

    # Get the list of variable types
    temp_var_list = milp_input.var_dict.keys()

    # Extract variable values for each variable type
    for temp_var in temp_var_list:
        # Filter variables of the current type from the model
        var_value = [var for var in model.variables() if temp_var in var.name]

        # Extract variable information and values
        var_value_list = [extract_pulp_var_info(x.name) + [x.varValue] for x in var_value]

        # Create a DataFrame for the variable type if data exists
        if var_value_list:  # Check if list is not empty
            df_dict[temp_var] = pd.DataFrame(data=var_value_list, columns=cols_gurobi_var[temp_var])

    return df_dict


def print_results(model, df_flight_flown, df_flight_canceled):
    """
    Prints the results of the optimization model and flight statistics.

    Args:
        model (pulp.LpProblem or list or float): The optimization model or a list of models.
        df_flight_flown (pd.DataFrame): DataFrame of flown flights with selected aircraft.
        df_flight_canceled (pd.DataFrame): DataFrame of canceled flights.

    Returns:
        None
    """
    if model is not None:
        print('---------------------   MODEL   ----------------')

        # Handle different types of models
        if isinstance(model, (list, np.float64)):
            if isinstance(model, np.float64):
                print("Model real")
                print("Model objective: {:.3e}".format(model))
                print("Model runtime: {:.3f}s".format(0))
            else:
                total_obj = 0
                for i, model_temp in enumerate(model):
                    print("-" * 20)
                    print("Model {}.Stage".format(i))
                    print("Model objective: {:.3e}".format(pulp.value(model_temp.objective)))
                    total_obj += pulp.value(model_temp.objective)
                print("-" * 20)
                print("Models objective: {:.3e}".format(total_obj))
        else:
            print("Model extensive")
            print("Model objective: {:.3e}".format(pulp.value(model.objective)))

        # Calculate and print flight cancellation statistics
        num_cancelled_flights = df_flight_canceled.shape[0]
        perc_cancelled_flights = round(
            num_cancelled_flights / (num_cancelled_flights + df_flight_flown.shape[0]) * 100, 1
        )

        print('---------------------   TA   ----------------')
        print('Total Scheduled flights: {}'.format(df_flight_flown['flight_identifier'].shape[0]))
        print('Cancelled flights (Datetime + Flight Number): {}'.format(
            df_flight_canceled['flight_identifier'].to_list()
        ))
        print("Total flights cancelled {} is {}% of all flights".format(
            num_cancelled_flights, perc_cancelled_flights
        ))


# Step 6: Input Data and Initialization

The input data for the problem is defined, including the fleet, flight schedule, and cost parameters.
The fleet and flight schedule are processed to generate the required dataframes for the optimization model.

# <font color='red'> ONLY CHANGE DATA IN THE CELL BELOW</font>

In [ ]:
# Input Configuration
input = {
    # Run Information
    'run': dt.datetime.now().strftime("%Y%m%d_%H%M"),

    # Time Settings
    'start_date': dt.datetime.strptime("2024-01-01", '%Y-%m-%d'),
    'end_date': dt.datetime.strptime("2024-01-04", '%Y-%m-%d'),

    # Aircraft Selection
    'aircraft_registration_selection': ["HBJBA", "HBJBB"], #"HBJBC", "HBJBD", "HBJBE"], #"HBJNA", "HBJNB", "HBJNC",

    # Cost Parameters
    'cost': {
        "C_ground": 10,  # Ground cost weight
        "C_cancel": 100000,  # Cancellation cost
        "C_overbooked_seat": 200,  # Overbooked seat cost
        "C_empty_seat": 10,  # Empty seat cost
        "C_fuel": 1, # Cost of fuel (per liter)
    }
}

# Automatically calculate derived parameters
input['n_aircraft'] = len(input['aircraft_registration_selection'])
input['n_days'] = (input['end_date'] - input['start_date']).days

## Step 7: Load Flight Schedule

The flight schedule is loaded and filtered based on the selected aircraft and time window.
The fleet dataframe is generated, and cost dataframes for ground and operating costs are prepared.

In [ ]:
import gdown

# Define file ID and output filename
file_id = "1cXSsUu9Ml71ST3-MeYu_IQWScTqN7ilK"
output_file = "flight_schedule.pkl"  # Change the filename if needed

# Google Drive download URL
gdrive_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(gdrive_url, output_file, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1cXSsUu9Ml71ST3-MeYu_IQWScTqN7ilK
To: /content/flight_schedule.pkl
100%|██████████| 4.96M/4.96M [00:00<00:00, 96.3MB/s]


'flight_schedule.pkl'

In [ ]:
flight_schedule = pd.read_pickle("flight_schedule.pkl")
print(f"All available aircraft registrations:")
print(np.unique(flight_schedule['aircraft_registration']))

# Filter flight_schedule according input
flight_schedule = flight_schedule[flight_schedule['aircraft_registration'].isin(input['aircraft_registration_selection'])]
flight_schedule = flight_schedule[(
    flight_schedule['departure_datetime_schedule_utc_dis']>= input['start_date'])
  & (flight_schedule['arrival_datetime_schedule_utc_dis']< input['end_date'])]
flight_schedule['possible_aircraft'] = flight_schedule['possible_aircraft'].apply(
    lambda x: [aircraft for aircraft in x if aircraft in input['aircraft_registration_selection']]
)
flight_schedule.reset_index(inplace=True)
display(flight_schedule.head())

# Gernerate fleet dataframe
aircraft_dataframe = get_aircraft_dataframe(flight_schedule)
fleet = Fleet(aircraft_dataframe)

# Get cost dataframes
ground_cost = get_ground_cost(fleet, input, flight_schedule['departure_airport_actual'].unique())
flight_cost = get_operating_cost(fleet, flight_schedule, input)

All available aircraft registrations:
['HBJBA' 'HBJBB' 'HBJBC' 'HBJBD' 'HBJBE' 'HBJBF' 'HBJBG' 'HBJBH' 'HBJBI'
 'HBJCA' 'HBJCB' 'HBJCC' 'HBJCD' 'HBJCE' 'HBJCF' 'HBJCG' 'HBJCH' 'HBJCI'
 'HBJCJ' 'HBJCK' 'HBJCL' 'HBJCM' 'HBJCN' 'HBJCO' 'HBJCP' 'HBJCQ' 'HBJCR'
 'HBJCS' 'HBJCT' 'HBJCU' 'HBJNA' 'HBJNB' 'HBJNC' 'HBJND' 'HBJNE' 'HBJNF'
 'HBJNG' 'HBJNH' 'HBJNI' 'HBJNJ' 'HBJNK' 'HBJNL']


,index,departure_airport_actual,arrival_airport_actual,departure_datetime_schedule_utc_dis,arrival_datetime_schedule_utc_dis,aircraft_registration,aircraft_sub_type,flight_duration,flight_duration_dec_hour,flight_identifier,total_pax,departure_datetime_schedule_uts_dis_int,arrival_datetime_schedule_utc_dis_int,possible_aircraft
0,5271,ZRH,CDG,2024-01-02 06:30:00,2024-01-02 08:00:00,HBJBB,221,0 days 01:30:00,1.5,240102LX632,105,122.0,128.0,"[HBJBA, HBJBB]"
1,5279,CDG,ZRH,2024-01-02 08:45:00,2024-01-02 10:15:00,HBJBB,221,0 days 01:30:00,1.5,240102LX633,91,131.0,137.0,"[HBJBA, HBJBB]"
2,5333,ZRH,AMS,2024-01-01 11:30:00,2024-01-01 13:00:00,HBJBB,221,0 days 01:30:00,1.5,240101LX728,80,46.0,52.0,"[HBJBA, HBJBB]"
3,5334,ZRH,AMS,2024-01-02 11:30:00,2024-01-02 13:00:00,HBJBB,221,0 days 01:30:00,1.5,240102LX728,105,142.0,148.0,"[HBJBA, HBJBB]"
4,5335,ZRH,AMS,2024-01-03 11:30:00,2024-01-03 13:00:00,HBJBA,221,0 days 01:30:00,1.5,240103LX728,98,238.0,244.0,"[HBJBA, HBJBB]"


## Step 8: Generate the Problem Instance

An instance of the `TAProblem` class is created using the input data, fleet, flight schedule, and cost dataframes.

In [ ]:
tap = TAProblem(input, fleet, flight_schedule, flight_cost, ground_cost)

## Step 9: Solve the Problem

The extensive form of the optimization model is solved using the `solve_extensive` method.
Solver parameters such as gap, time limit, and number of threads are specified.

In [ ]:
model = tap.solve_extensive(gap=0.02, time_limit=600)

## Step 10: Analyze the Results

The model status is checked to ensure the solution is feasible.
If feasible, the results are extracted, including flown and canceled flights, and visualized using the plotting functions.

In [ ]:
if pulp.LpStatus[model.status] != 'Infeasible' and pulp.LpStatus[model.status] != 'Not Solved':
    file_add = input['run']
    res = get_flight_stats(tap, flight_schedule)
    df_flight_flown, df_flight_canceled, df_flight_counter = res
    df_dict_res = get_model_stats(tap, model=model)

    print_results(model, df_flight_flown, df_flight_canceled)

---------------------   MODEL   ----------------
Model extensive
Model objective: 8.978e+04
---------------------   TA   ----------------
Total Scheduled flights: 33
Cancelled flights (Datetime + Flight Number): []
Total flights cancelled 0 is 0.0% of all flights


In [ ]:
plot_aircraft_schedules_plotly(input, df_flight_flown, file_add)

In [ ]:
plot_flightplan_plotly(input, df_flight_flown, df_flight_canceled, file_add, colormap='Dark2')

# Appendix
Example for Dynamic Programming: Fibonacci

In [ ]:
def fibonacci_dp(n):
    # Base cases
    if n == 0:
        return 0
    if n == 1:
        return 1

    # Create a table to store Fibonacci numbers
    dp = [0] * (n + 1)
    dp[0] = 0
    dp[1] = 1

    # Fill the table iteratively
    for i in range(2, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]

    return dp[n]

# Example usage
n = 5
print(f"The {n}th Fibonacci number is: {fibonacci_dp(n)}")

The 5th Fibonacci number is: 5
